In [148]:
from langchain_core.tools import tool
from langchain_cohere import ChatCohere
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage,SystemMessage
import requests

Tool Creation

In [149]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency:str,target_currency:str) ->float :
    """This function fetched the currency conversion factor between a given base currency and target currency"""
    url=f"https://v6.exchangerate-api.com/v6/2aa873ed1233af2d9c517593/pair/{base_currency}/{target_currency}"
    response=requests.get(url)
    return response.json()

@tool
def convert(base_currency_value:int, conversion_rate:Annotated[float,InjectedToolArg])->float:
    """Given a currency conversion rate ,this function calculates the target currency from a given base currency value"""
    return base_currency_value*conversion_rate

In [150]:
get_conversion_factor.invoke({"base_currency":"USD","target_currency":"INR"})["conversion_rate"]

89.2014

Tool Binding

In [151]:
load_dotenv()
llm=ChatCohere(model="command-a-03-2025")

In [152]:
llm_with_tools=llm.bind_tools([get_conversion_factor,convert])

In [153]:
# messages=[HumanMessage("What is the conversion factor between USD to INR ,and based on that can you convert 10 USD to INR?")]
system_msg = SystemMessage(
    content=(
        "You are allowed to output multiple tool calls in a single response when appropriate. "
        "If you need to compute something that requires two steps (for example: fetch a conversion rate, "
        "then convert an amount), you may output BOTH tool calls in the same response. "
        "If you do not know an exact numeric value (like the conversion rate), you may include a reasonable "
        "estimate in the second tool call argument named 'conversion_rate'. Mark estimates clearly in text if possible. "
        "Always provide the plan and the sequence of tool calls before the tool calls."
    )
)

user = HumanMessage(content="Find the conversion factor between USD and INR, and using that, convert 10 USD to INR.")


In [154]:
messages=[system_msg,user]

In [155]:
ai_messages=llm_with_tools.invoke(messages)
messages.append(ai_messages)

In [156]:
ai_messages.tool_calls  

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'get_conversion_factor_he42ffxcnvce',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': 'convert_6e3gtzbd9vfv',
  'type': 'tool_call'}]

In [157]:
import json

for tool_call in ai_messages.tool_calls:
    #execute the 1st tool and get the conversion rate
    if tool_call["name"]=="get_conversion_factor":
        tool_message1=get_conversion_factor.invoke(tool_call)
        # print(tool_message1)
        #now we need to fetch the conversion rate and then append this tool_message1 to messages
        
        conversion_rate=json.loads(tool_message1.content)["conversion_rate"]
        messages.append(tool_message1)
        
    #execute the tool2 using the conversion rate fetched from tool 1

    if tool_call["name"]=="convert":
        #fetch the current args and add/update conversion rate there
        tool_call["args"]["conversion_rate"]=conversion_rate
        tool_message2=convert.invoke(tool_call)
        messages.append(tool_message2)

In [158]:
messages

[SystemMessage(content="You are allowed to output multiple tool calls in a single response when appropriate. If you need to compute something that requires two steps (for example: fetch a conversion rate, then convert an amount), you may output BOTH tool calls in the same response. If you do not know an exact numeric value (like the conversion rate), you may include a reasonable estimate in the second tool call argument named 'conversion_rate'. Mark estimates clearly in text if possible. Always provide the plan and the sequence of tool calls before the tool calls.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Find the conversion factor between USD and INR, and using that, convert 10 USD to INR.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='I will first find the conversion factor between USD and INR. Then, I will use that factor to convert 10 USD to INR.', additional_kwargs={'id': 'b7a528eb-34ec-4c04-91b7-ec9a96a7f338', 'finish_reason': 'TOOL

In [159]:
llm_with_tools.invoke(messages).content

'The conversion factor between USD and INR is 89.2014.\n\nUsing this factor, 10 USD is equal to 892.0140000000001 INR.'

This is not AI agent as we needed to do hardcore the logics instead of being autonomous